# Python file set up notebook

1. get a list of all the obsids we're analyzing and write this to a file

In [ ]:
def obsidIterator(obsidFolder, listDestFolder):
    """iterate through all obsids in a folder containing raw PPS products"""
    """return all obsids of interest and all source region file names"""
    curdir = os.getcwd()

    if os.path.isdir(obsidFolder):
        os.chdir(obsidFolder)
        subprocess.call(['ls > {0}/obsids.txt'.format(listDestFolder)],shell=True)
        with open('{0}/obsids.txt'.format(listDestFolder)) as f:
            reader = csv.reader(f)
            for row in reader:
                currObsid = row[0]
                ppsPath = obsidFolder + '/' + currObsid + '/PPS'
                os.chdir(ppsPath)
                subprocess.call(['ls *SRCREG* > {0}/{1}_srcreglist.txt'.format(listDestFolder,currObsid)],shell=True)
    else:
        print("OBSID folder does not exist as entered")

2. for each obsid generated above, we need to get all the region files. Saving a copy in physical (for SAS calls) and in FK5 RA/DEC (for meta file later) and writing this out. Leaving them in DS9 format in case we want to get a pretty picture.

In [51]:
import csv
obsid = '0810880201'
obsidFolder = '/home/kirk/Documents/research/XMM_Newton/obsids'

def getSrcReg(obsid,obsidFolder):
    with open('{0}_srcreglist.txt'.format(obsid)) as f1:
        reader1 = csv.reader(f1)
        for row in reader1:
            cam = row[0][11:13] #either M1, M2, PN (assuming file name size is static)
            if cam == 'M1' or cam == 'M2' or cam == 'PN':
                filename=obsidFolder + '/' + obsid + '/PPS/' + row[0]
                srcString = row[0][17:-4] #ie SRCREG0001
                lines = open(filename).read().splitlines()
                with open('{0}_{1}_{2}_PHYS.reg'.format(obsid,cam,srcString),'w') as outFile:
                    outFile.write("# Region file format: DS9 version 4.1\n")
                    outFile.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')           
                    outFile.write("physical\n")
                with open('{0}_{1}_{2}_FK5.reg'.format(obsid,cam,srcString),'w') as outFile:
                    outFile.write("# Region file format: DS9 version 4.1\n")
                    outFile.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')           
                    outFile.write("FK5\n")
                for line in lines:
                    #print(line[:8])
                    if line[:8] == 'detector':
                        for i in range(8,len(line)):
                            if line[i] == 'c' and line[i+1]=='i':
                                with open('{0}_{1}_{2}_PHYS.reg'.format(obsid,cam,srcString),'a') as outFile:
                                    outFile.write("{0}\n".format(line[i:]))
                                break
                    elif line[:3] == 'fk5':
                        for i in range(3,len(line)):
                            if line[i] == 'c' and line[i+1]=='i':
                                with open('{0}_{1}_{2}_FK5.reg'.format(obsid,cam,srcString),'a') as outFile:
                                    outFile.write("{0}\n".format(line[i:]))
                                break
            else:
                print("something is wrong -- file size not static?")
         

In [52]:
getSrcReg(obsid,obsidFolder)

3. to use SAS commands we need just the circles -- this function returns just the circle part of the files output above. 

In [63]:
testPHYS = '0810880201_M2_SRCREG0005_PHYS.reg'
testFK5 = '0810880201_M2_SRCREG0005_FK5.reg'
def getCircles(regListFile):
    src=open(regListFile)
    srcregions=src.readlines()
    src.close()
    srcregions=[src.strip() for src in srcregions[3:]] #start at 3 to discard header
    circles = []
    for i in range(len(srcregions)):
        for j in range(len(srcregions[i])):
            if srcregions[i][j] == ")": #stop at second paranthetical
                circles.append(srcregions[i][:j+1]) #append just circle portion of text file
                break
    return circles 

In [64]:
getCircles(testPHYS)

['circle(17561, 23321, 340)',
 'circle(13638.3, 22299.5, 3360)',
 'circle(15485, 21080, 530)',
 'circle(15767, 21829, 340)',
 'circle(15285, 24808, 260)']

In [67]:
getCircles(testFK5) #not sure why there are more in the FK5 format than in the detector cooordinate but whatever who fucking cares?

['circle( 16.007030,-72.027566,60")',
 'circle( 16.007073,-72.032562,60")',
 'circle( 15.906240,-72.026027,60")',
 'circle( 16.290637,-72.196650,60")',
 'circle( 16.479233,-72.064111,60")',
 'circle( 16.344486,-71.947096,60")',
 'circle( 15.777896,-71.861375,60")',
 'circle( 16.570436,-72.093886,60")',
 'circle( 16.557420,-72.083508,60")',
 'circle( 16.089720,-72.049465,60")',
 'circle( 16.401971,-72.163669,60")',
 'circle( 15.403895,-72.071995,60")',
 'circle( 15.973363,-72.152145,60")',
 'circle( 15.899363,-72.056381,60")',
 'circle( 15.703647,-71.966921,60")',
 'circle( 15.429444,-71.908387,60")',
 'circle( 15.820675,-72.158116,60")',
 'circle( 16.386912,-72.225467,60")',
 'circle( 16.417030,-72.124351,60")',
 'circle( 16.389949,-72.185581,60")',
 'circle( 16.232239,-72.171797,60")',
 'circle( 16.106088,-72.237652,60")',
 'circle( 15.993584,-71.997755,60")',
 'circle( 16.578121,-72.042065,60")',
 'circle( 15.408000,-71.904223,60")']

**NOTE** any cells (like several below) that make use of heasoft/SAS tools don't work in the notebook and I don't care to figure out how to make them work but I'm leaving them here to better document thought process.

4. For a given circle, we want to be able to extract a LC. 

In [68]:
def lcextraction(abin, arange, emin, emax, table, srcregion, camera,
                 tstart, tstop, OBSID, srcString, iter):
    ''' Extract a barycentric corrected lightcurve
        in the energy range['emin':'emax'], with time bins of 'bin'
    '''
    #srclc = "0810880201_M1_lc_SRCREG0001_1_0310keV_bin1.ds"
    srclc = "{0}_{1}_lc_{2}_{3}_{4}keV_bin{5}.ds".format(OBISD, cam, srcString, iter, arange, abin)
    srcimg = "{0}_{1}_img_{2}_{3}_{4}keV_bin{5}.ds".format(OBISD, cam, srcString, iter, arange, abin)
    psimg = "{0}_{1}_lcPlot_{2}_{3}_{4}keV_bin{5}.ps".format(OBISD, cam, srcString, iter, arange, abin)

    if cam == 'PN':
        srcexp = 'expression=#XMMEA_EP && (PI IN [{0}:{1}]) && PATTERN <=4\
 && FLAG==0 && ((X,Y) IN {2})'.format(emin, emax, srcregion)

    elif cam == 'M1':
        srcexp = 'expression=#XMMEA_EM && (PI IN [{0}:{1}]) && PATTERN <=12\
 && FLAG==0 && ((X,Y) IN {2})'.format(emin, emax, srcregion)

    elif cam == 'M2':
        srcexp = 'expression=#XMMEA_EM && (PI IN [{0}:{1}]) && PATTERN <=12\
 && FLAG==0 && ((X,Y) IN {2})'.format(emin, emax, srcregion)

    else:
        print ("Something is wrong, the camera doesn't exist")
        raw_input("Please press 'Ctrl-C' to terminate and check for errors")

    subprocess.call(
        ['evselect', 'table={0}'.format(table),'energycolumn=PI','rateset={0}'.format(srclc),
        'timebinsize={0}'.format(abin),'maketimecolumn=yes','makeratecolumn=yes',
        'imageset={0}'.format(srcimg),'xcolumn=X', 'ycolumn=Y','ximagebinsize=80','yimagebinsize=80',
        'timemin={0}'.format(tstart), 'timemax={0}'.format(tstop),srcexp] #testing simple version
    )

    subprocess.call(
        ['dsplot', 'table={0}'.format(srclc), 'withx=yes', 'withy=yes',
         'x=TIME', 'y=RATE',
         'plotter=xmgrace -hardcopy -printfile {0}'.format(psimg)])

    return True

5. We need to get some parameters for the LC function with some other functions (ie start and stop times).

In [70]:
def findtimes(rpcdata_dir, camera):
    """Find the initial and final times of observation for the given camera"""
    currentDir = os.getcwd()
    os.chdir(rpcdata_dir)
    #print(os.getcwd())
    evtfile = glob.glob('*{0}*ImagingEvts.ds'.format(camera.upper()))
    #print(evtfile
    tstart = fits.getval(evtfile[0], 'TSTART', ext=1)
    tstop = fits.getval(evtfile[0], 'TSTOP', ext=1)
    os.chdir(currentDir)
    return tstart, tstop

6. Need to take resulting fits LC file and output to CSV

In [85]:
testDump = 'mos1_lc_src_0310keV_bin1_35.ds'
#note: real file will be like:
#srclc = "0810880201_M1_lc_SRCREG0001_1_0310keV_bin1.ds"
import subprocess
def lc2CSV(infile):
    outfile = infile[:-3] + '.csv' #change the extension from .ds to .csv
    subprocess.call(['fdump {0}[1] {1} rows=- columns=- prhead=no, fldsep=","'.format(infile, outfile)],shell=True)
    #don't print the header, print all the rows and columns from the rate index

# FFT Processing and master file writing function done in Julia notebook

overall process below!!!!

In [ ]:
#1 get obsids
obsid = '0810880201'
obsidFolder = '/home/kirk/Documents/research/XMM_Newton/obsids'
listDestFolder = '/home/kirk/Documents/research/XMM_Newton/python_xmmscripts/pipeline'
obsidIterator(obsidFolder, listDestFolder)
#2 get srcreg files LOOP (i)
getSrcReg(obsid,obsidFolder)
#3 get detector circles for SAS commands LOOP (j)
getCircles(testPHYS)
#4 set up needed info for lcextraction LOOP (j)
emin=300
emax=2000
table='mos1/events/evts_barycen.ds'
cam="mos1"
abin=1 #1second lc
ranges = '0310' #['0310', '032', '245', '4510', '210']
tstart, tstop = findtimes('/home/kirk/Documents/research/XMM_Newton/python_xmmscripts/pipeline/rpcdata/', cam)
#5 extract lc LOOP (j)
lcextraction(abin, arange, emin, emax, table, srcregion, camera,
                 tstart, tstop, OBSID, srcString, j)
#6 send lc to CSV LOOP (j)
infile="{0}_{1}_{2}_lc_src_{3}keV_bin{4}_{5}_{6}".format(obsid,cam,arange,abin,srcReg,j)
lc2CSV(infile)
#7 call Julia program to generate fft and allInfo list

In [89]:
test = open('obsids.txt')
x = test.readlines()
test.close()

In [97]:
ls

0810880201_M1_SRCREG0001_FK5.reg   mos1_lc_net_0310keV_bin1_6.ps
0810880201_M1_SRCREG0001_PHYS.reg  mos1_lc_net_0310keV_bin1_7.ps
0810880201_M1_SRCREG0002_FK5.reg   mos1_lc_net_0310keV_bin1_8.ps
0810880201_M1_SRCREG0002_PHYS.reg  mos1_lc_net_0310keV_bin1_9.ps
0810880201_M1_SRCREG0003_FK5.reg   mos1_lc_net_0310keV_bin1.ps
0810880201_M1_SRCREG0003_PHYS.reg  mos1_lc_src_0310keV_bin1_0.ds
0810880201_M1_SRCREG0004_FK5.reg   mos1_lc_src_0310keV_bin1_10.ds
0810880201_M1_SRCREG0004_PHYS.reg  mos1_lc_src_0310keV_bin1_11.ds
0810880201_M1_SRCREG0006_FK5.reg   mos1_lc_src_0310keV_bin1_12.ds
0810880201_M1_SRCREG0006_PHYS.reg  mos1_lc_src_0310keV_bin1_13.ds
0810880201_M1_SRCREG0007_FK5.reg   mos1_lc_src_0310keV_bin1_14.ds
0810880201_M1_SRCREG0007_PHYS.reg  mos1_lc_src_0310keV_bin1_15.ds
0810880201_M1_SRCREG000A_FK5.reg   mos1_lc_src_0310keV_bin1_16.ds
0810880201_M1_SRCREG000A_PHYS.reg  mos1_lc_src_0310keV_bin1_17.ds
0810880201_M1_SRCREG0013_FK5.reg   mos1_lc_src_0310keV_bin1_18.ds
0810880201_M1_SRCR

In [99]:
test = '0810880201_M1_SRCREG0001_PHYS.reg'
test[-8:-4]

'PHYS'

In [101]:
test.split("_")[1]

'M1'

In [105]:
test=test[:-8]+'FK5.reg'

In [121]:
f = open('{0}'.format(test))
x = f.readlines()
f.close()
raDec = x[3].split(",")

In [130]:
raDec[0].split()

['circle(', '16.007030']

In [128]:
raDec[1]

'-72.027566'

In [134]:
x[3]

'circle( 16.007030,-72.027566,60") # color=cyan\n'

In [135]:
import pandas as pd
cols = ['rate','error','time']
df = pd.DataFrame(columns = cols)

In [136]:
df = df.fillna(0) 

In [203]:
import pandas as pd
import numpy as np
def csv2DF(csvFile):
    rate =[]
    time =[]
    error =[]
    with open(csvFile) as f1:
        reader1 = csv.reader(f1)
        rowCount = 1
        for row in reader1:
            if rowCount > 4:
                rate.append(float(row[1]))
                error.append(float(row[2]))
                time.append(float(row[3]))
            rowCount+=1
    df = pd.DataFrame({'rate' : np.array(rate), 'time' : np.array(time), 'error' : np.array(error)})
    return df

In [204]:
csvFile = "0810880201_M1_lc_SRCREG000C_0_0310keV_bin1.csv"
df = csv2DF(csvFile)

In [205]:
df

,rate,time,error
0,0.0,6.571532e+08,0.0
1,0.0,6.571532e+08,0.0
2,0.0,6.571532e+08,0.0
3,0.0,6.571532e+08,0.0
4,0.0,6.571532e+08,0.0
5,0.0,6.571532e+08,0.0
6,0.0,6.571532e+08,0.0
7,0.0,6.571532e+08,0.0
8,0.0,6.571532e+08,0.0
9,0.0,6.571532e+08,0.0


In [180]:
rate = np.array(rate)
time = np.array(time)
error = np.array(error)
pd.DataFrame({'rate' : rate, 'time' : time, 'error' : error})

,rate,time,error
0,0.000000000000000E+00,6.571531956661710E+08,0.000000000000000E+00
1,0.000000000000000E+00,6.571531966661710E+08,0.000000000000000E+00
2,0.000000000000000E+00,6.571531976661710E+08,0.000000000000000E+00
3,0.000000000000000E+00,6.571531986661710E+08,0.000000000000000E+00
4,0.000000000000000E+00,6.571531996661710E+08,0.000000000000000E+00
5,0.000000000000000E+00,6.571532006661710E+08,0.000000000000000E+00
6,0.000000000000000E+00,6.571532016661710E+08,0.000000000000000E+00
7,0.000000000000000E+00,6.571532026661710E+08,0.000000000000000E+00
8,0.000000000000000E+00,6.571532036661710E+08,0.000000000000000E+00
9,0.000000000000000E+00,6.571532046661710E+08,0.000000000000000E+00


In [284]:
def genFFTDF(df):
    fftOut=np.fft.fft(df.rate[:]) #rate is really counts (counts/s recorded every sec)
    fftOut=fftOut[1:] #get rid of Nyqhuist
    fftOut=fftOut[:len(fftOut)//2] #take half (rounding midpoint down if needed)
    fftOut=np.abs(fftOut) #square magnitude
    fftOut=fftOut**2
    print(fftOut)
    avgPow=np.mean(fftOut)
    fftOut=fftOut/avgPow #normalize
    dt=df.time[len(df.time[:])-1]-df.time[0] #for some reason -1 indexing wasn't working here whatever
    freq=[i/dt for i in range(len(fftOut))] #freq(i) = bin#/totalTime
    fftDF=pd.DataFrame({'freq' : freq, 'pow' : fftOut})
    return freq,fftOut

In [220]:
def createAllInfo(powCutoff,obsid,cam,srcReg,srcNum,startdate,ra,dec,fitsDF,fftInfo):
    outFile = "xmm.fftinfo.all{0}".format(powCutoff)
    exptime = fitsDF.time[-1]-fitsDF.time[0]
    obsidCam = "{0}_{1}".format(obsid,cam)
    srcRegNum = "{0}_{1}".format(srcReg,srcNum)
    with open(outFile,'a') as f:
        for i in range(len(fftInfo.pow[:])):
            if fftInfo.pow[i] >= powCutoff:
                freq = fftInfo.f[i]
                power = fftInfo.pow[i]
                f.write("{0},{1},{2},{3},{4},{5},{6},{8}\n".format(obsidCam,srcRegNum,startdate,exptime,ra,dec,freq,power))
        

In [224]:
def saveFFT(obsid,cam,srcReg,srcNum,fftDF):
    exportFile="{0}_{1}_{2}_{3}_FFT.csv".format(obsid,cam,srcReg,srcNum)
    fftDF.to_csv(exportFile,index=False)

In [285]:
freq,power = genFFTDF(df)
saveFFT(12,12,12,12,fftDF)

[136.8773792   13.78736925  27.31032434 ... 178.5218953   78.24757285
  38.49543253]


In [226]:
fin = open("data.txt", "a")

fin.write('\nThis is newly appended text.');

fin.close()

In [268]:
x=list(fftDF.columns.values)

In [289]:
fftDF=pd.DataFrame({'freq' : freq, 'power' : power})

In [290]:
fftDF.power

0        1.116232
1        0.112436
2        0.222715
3        0.692803
4        1.881577
5        0.041101
6        0.234330
7        2.888469
8        0.159815
9        0.595730
10       0.358325
11       0.222215
12       0.252278
13       1.091940
14       0.953882
15       0.473298
16       0.320114
17       0.166201
18       2.475512
19       3.270767
20       1.831234
21       0.923907
22       0.353058
23       1.398756
24       0.064921
25       1.498111
26       0.051967
27       1.687824
28       0.729708
29       0.393316
           ...   
18031    0.044852
18032    0.682718
18033    1.027982
18034    1.045906
18035    1.770554
18036    0.649225
18037    4.356250
18038    0.441890
18039    0.156771
18040    1.527663
18041    1.167252
18042    1.341573
18043    0.126809
18044    2.663408
18045    0.108089
18046    1.511468
18047    1.461791
18048    0.490268
18049    0.849381
18050    0.049889
18051    0.073748
18052    0.844902
18053    0.188124
18054    0.960678
18055    1